In [40]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import transformers
from torch.optim import Adam
from tqdm import tqdm

In [41]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

In [42]:
from utils import FinQA_Dataset

In [43]:
TOKENIZER = T5Tokenizer.from_pretrained("t5-base")
MODEL = T5ForConditionalGeneration.from_pretrained("../finqa_finetune_t5.pth/")
BATCH_SIZE = 8
TESTING = True

/Users/ashish1610dhiman/opt/anaconda3/envs/dlt_project/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [44]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

### Read Test and do inference

In [45]:
finqa_test = torch.load('../finqa_dataset/finqa_test.pth')
len(finqa_test)

1108

In [46]:
if TESTING:
    test_loader = DataLoader(Subset(finqa_test, range(100)), batch_size=BATCH_SIZE)
else:
    test_loader = DataLoader(finqa_test, batch_size=BATCH_SIZE)

In [47]:
TOKENIZER.decode([x for x in finqa_test[0]['labels'] if x!= -100], skip_special_tokens=True)

'Step 1: Subtract 5735 from 5829. This gives the result: 94'

In [48]:
MODEL.eval();
generated_texts = []
for batch in tqdm(test_loader, desc="Generation batches"):
    input_ids = batch["input_ids"].to(DEVICE)
    attention_mask = batch["attention_mask"].to(DEVICE)
    decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)    
    with torch.no_grad():
        # Use the `generate` method for text generation
        generated_output = MODEL.generate(
            input_ids=input_ids,
            max_length=128,  # Set the desired maximum length for generated text
            num_beams=5,    # You can adjust the number of beams for beam search
            temperature=0.8  # You can adjust the temperature for sampling,
            )
        generated_text = TOKENIZER.decode([x for x in generated_output[0] if x!= -100], skip_special_tokens=True)
        generated_texts.append(generated_text)

Generation batches:   0%|          | 0/13 [00:00<?, ?it/s]

Generation batches: 100%|██████████| 13/13 [22:12<00:00, 102.49s/it]


In [49]:
generated_texts

['Step 1: Subtract 5735 from 5829. This gives the result: 350',
 'Step 1: Check if 153.55 is greater than 198.09. This gives the result: yes',
 'Step 1: Add 705.4 and 703.1. This gives the result: 177.9 ####### Step 2: Add 177.9 and const_2. This gives the result: 177.9 ####### Step 3: Divide 177.9 by const_2. This gives the result: 705.4',
 'Step 1: Divide 219000 by 83000. This gives the result: 1.1',
 'Step 1: Subtract 61912 from 51052. This gives the result: 61912',
 'Step 1: Multiply 160196 by 58.02. This gives the result: 2082082.5 ####### Step 2: Multiply 1601962.5 by const_1000000. This gives the result: 1601965000000 ####### Step 3: Multiply 1601965000000 by 52.35. This gives the result: 20800000000 ####### Step 4: Divide 20800000000 by const_1000. This gives the result: 20800000000 ####### Step 5: Multiply 160196 by 58.02. This gives',
 'Step 1: Subtract 201 from 189. This gives the result: -12 ####### Step 2: Divide -12 by 201. This gives the result: -4%',
 'Step 1: Subtract 

In [ ]:
import pickle
file_path = 'generated_sentences_test.pkl'
# Open the file in binary read mode and use pickle.load() to load the list
with open(file_path, 'wb') as file:
    pickle.dump(generated_texts, file)